# ***Dataset Preprocessing***

This notebook shows the preprocessing of the dataset.

### ***Import packages***

Before we begin, let's import all the necessary packages for this notebook.
First we add the directory which has our python files:

In [1]:
import sys
sys.path.insert(0, "../..")

Next we import all the necessary packages for this notebook:

In [2]:
import pandas as pd
from os.path import join
from pathlib import Path
from pyimzml.ImzMLParser import ImzMLParser
from pyimzml.ImzMLWriter import ImzMLWriter
from tqdm.notebook import tqdm
from nnbiopsy.spectra_utils.normalization import TICNormalizer
from nnbiopsy.spectra_utils.binning import EqualWidthBinning

### ***Constants definitions***

Next, let's define some constant variables for this notebook:

In [3]:
# Define folder that contains the dhg dataset
DHG_PATH = "./../../../DHG/"
# Define folder that contains the original dataset
DATA_IN_PATH = f"{DHG_PATH}/raw"
# Define folder to save preprocessed dataset
DATA_OUT_PATH = f"{DHG_PATH}/preprocessed"
# Define file that contains dhg metadata
METADATA_PATH = f"{DHG_PATH}/metadata.csv"
# Define mass range start value
MZ_START = 50
# Define mass range end value
MZ_END = 1200
# Define mass resolution of the data
MASS_RESOLUTION = 0.025

### ***Creating output folder***

Next, let's create the output folder:

In [4]:
# Create output folder if doesn't exist
Path(DATA_OUT_PATH).mkdir(parents=True, exist_ok=True)

### ***Reading MSI metadata***

Next, lets read the metadata file:

In [5]:
# read ROI's csv
metadata_df = pd.read_csv(METADATA_PATH)

### ***Creating spectra preprocess pipeline***

Next, lets create the preprocess pipeline for each spectra in the image:

In [6]:
# Get normalizer object
normalizer = TICNormalizer()
# Get binning object
binning = EqualWidthBinning(MZ_START, MZ_END, MASS_RESOLUTION / 2)
# Create preprocess pipe
pre_process_pipe = (lambda mzs, intensities: (binning.bin(
    normalizer.normalize((mzs, intensities)))))

### ***Preprocess each MSI***

Next, let's preprocess each MSI spectra. For each sample (ROI) we extract the sample from the MSI, and for each ROI we preprocess all spectra's using spectra preprocess pipeline:

In [7]:
# loop over each ROI in dataframe
for index, roi in tqdm(metadata_df.iterrows(),
                       total=metadata_df.shape[0],
                       desc="ROIs Loop"):
  # Parse the MSI file containing ROI
  with ImzMLParser(join(DATA_IN_PATH, f"{roi.file_name}.imzML")) as reader:
    # Create a new MSI for ROI. because we apply binning
    # we can use mode="continuous"
    with ImzMLWriter(join(DATA_OUT_PATH,
                          f"{roi.sample_file_name}.imzML"),
                     mode="continuous") as writer:
      # Loop over each spectra in MSI
      for idx, (x, y, z) in tqdm(enumerate(reader.coordinates),
                                 total=len(reader.coordinates),
                                 desc="Spectra Loop"):
        # Check if spectra is in ROI boundaries
        if ((roi.x_min <= x) & (roi.x_max >= x) & (roi.y_min <= y) &
            (roi.y_max >= y)):
          # Read spectra from MSI
          raw_mzs, raw_intensities = reader.getspectrum(idx)
          # Apply preprocessing pipe
          preprocessed_mzs, preprocessed_intensities = pre_process_pipe(
              raw_mzs, raw_intensities)
          # Write spectra to ROI MSI but by ROIs coordinates
          writer.addSpectrum(preprocessed_mzs, preprocessed_intensities,
                             (x - roi.x_min + 1, y - roi.y_min + 1, z))

ROIs Loop:   0%|          | 0/48 [00:00<?, ?it/s]

Spectra Loop:   0%|          | 0/14022 [00:00<?, ?it/s]

Spectra Loop:   0%|          | 0/14022 [00:00<?, ?it/s]

Spectra Loop:   0%|          | 0/14022 [00:00<?, ?it/s]

Spectra Loop:   0%|          | 0/7482 [00:00<?, ?it/s]

...

Spectra Loop:   0%|          | 0/28584 [00:00<?, ?it/s]

Spectra Loop:   0%|          | 0/15045 [00:00<?, ?it/s]

Spectra Loop:   0%|          | 0/15045 [00:00<?, ?it/s]

Spectra Loop:   0%|          | 0/6000 [00:00<?, ?it/s]

Spectra Loop:   0%|          | 0/6000 [00:00<?, ?it/s]